In [ ]:
!python main.py

In [ ]:
# Only remake all .md posts
from scholar.post_maker import make_all

make_all(overwrite=True)

# Archive

Contains 1 off snippets to make change to the system.

## Add new column called arxiv_term in SQLite

In [ ]:
# import sqlite3

# Drop column
# with sqlite3.connect('scholar/paper.db') as conn:
#     c = conn.cursor()
#     c.execute('ALTER TABLE paper DROP COLUMN arxiv_group_tag;')
#     conn.commit()

# with sqlite3.connect("scholar/paper.db") as conn:
#     c = conn.cursor()
#     c.execute("ALTER TABLE paper ADD COLUMN category VARCHAR(255);")
#     conn.commit()

## Compute new column

In [ ]:
# import sqlite3
# import pandas as pd
# from scholar.database import Paper

# with sqlite3.connect("scholar/paper.db") as conn:

#     # Add journal column
#     cursor = conn.cursor()
#     cursor.execute("ALTER TABLE paper ADD COLUMN journal VARCHAR(255);")

#     # Export data to pandas for easier manipulation
#     df = pd.read_sql_query("SELECT * FROM paper", conn)


# # Extract journal name from publication_info_summary
# journals = df.publication_info_summary.str.split(" - ", expand=True)[2]

# # Update the database
# for record, new_value in zip(Paper.select(), journals):
#     record.journal = new_value
#     record.save()

## Patch arxiv

In [ ]:
Paper.delete().where(Paper.id == 389).execute()

In [ ]:
from scholar.database import Paper
from scholar.api import query_arxiv

papers = Paper.select().where(
    Paper.title == "Reallocating and Resampling: A Comparison for Inference"
)
for paper in papers:
    print(paper.title)
    print(paper.arxiv_id)

    paper.arxiv_id = "1708.02102"
    new_info = query_arxiv(paper.arxiv_id)

In [ ]:
from scholar.database import Paper
from scholar.api import query_arxiv, sanitize_title
import datetime

papers = Paper.select().where(Paper.published_on.year == 2000)

for paper in papers:
    print(paper.title)
    # print(paper.publication_info_summary)
    paper.published_on = datetime.datetime(2015, 10, 11)
    paper.save()
    # new_info = query_arxiv(paper.arxiv_id)
    # if new_info is not None:
    #     print(f"Updating {paper.id} {paper.title}")
    #     print(sanitize_title(new_info["title"]))
    #     paper.title = sanitize_title(new_info["title"])
    #     paper.authors = new_info["authors"]
    #     paper.arxiv_category_tag = new_info["arxiv_category_tag"]
    #     paper.category = new_info["category"]
    #     paper.published_on = new_info["published_on"]
    #     paper.save()

In [ ]:
len_dict

In [ ]:
import arxiv


search = arxiv.Search(id_list=["1811.02963"], max_results=1)
result = next(search.results())

In [ ]:
{
    "authors": ", ".join([str(author) for author in result.authors]),
    "doi": result.doi,
    "arxiv_category_tag": result.primary_category,
    "category": result.primary_category,
    "published_on": result.published,
    "title": result.title,
}

In [ ]:
result

In [ ]:
result.keys()

In [ ]:
repeated_journals = {k: v for k, v in all_journals.items() if v >= 3}
sorted_journals = sorted(repeated_journals.items(), key=lambda x: x[1], reverse=False)

In [ ]:
# Naive whitelist, count >= 3...
with open("scholar/whitelist_journals.txt", "w") as f:
    for key in repeated_journals.keys():
        f.write(key + "\n")

## Query SERP for older papers

Get top-1000 relevant papers = 50 queries

In [ ]:
from scholar.api import query_serp, query_arxiv, query_biorxiv
from scholar.database import insert_result

# term = '"simulation-based+inference"'
# next_url = None

# term = None
# next_url = "https://serpapi.com/search.json?as_sdt=0%2C5&as_vis=1&as_ylo=2000&engine=google_scholar&hl=en&num=20&q=%22simulation-based%2Binference%22&start=100"

# for _ in range(5):
#     results = query_serp(url=next_url, term=term, historical=True)

#     for result in results["formatted_results"]:
#         # Append extra arxiv data
#         if result["journal"] == "arxiv.org":
#             arxiv_data = query_arxiv(result["arxiv_id"])
#             if arxiv_data is not None:
#                 result.update(arxiv_data)

#         # Append extra biorxiv data
#         if result["journal"] == "biorxiv.org":
#             biorxiv_data = query_biorxiv(result["doi"])
#             if biorxiv_data is not None:
#                 result.update(biorxiv_data)

#         # Insert into database
#         insert_result(result)

#         try:
#             next_url = results["serpapi_pagination"]["next"]
#             term = None
#         except KeyError:
#             break